## Demo 6 - Moduli space of unirateral equilateral pentagons 

### A toy model for energy landscapes and molecular dynamics

In [ ]:
import random
import numpy as np

#manifold learning and dim red
from sklearn.manifold import Isomap
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding as LLE

#topological data analysis
from ripser import ripser
from persim import plot_diagrams
from dreimac import CircularCoords, CircleMapUtils

#plotting and visualization
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, no_update
import plotly.graph_objects as go
import pandas as pd
import base64

from matplotlib import cm
import matplotlib.pyplot as plt
%matplotlib inline

# Moduli Space of Equilateral Pentagons

The space of planar pentagons of side length 1 is the set


$$\mathbb{M} = \left\{ (\mathbf{p}_1, \ldots, \mathbf{p}_5) \in \mathbb{C}^5 \; : \; |\mathbf{p}_{i} - \mathbf{p}_{i+1}| = 1 \right\} \big/ \sim $$

where two pentagons are regarded as equivalent, $\mathbf{p} \sim  \mathbf{p}' $, if and only if $ \mathbf{p}' = U\mathbf{p} + \mathbf{v} $ for   $ U \in U(1) $ and $ \mathbf{v} \in \mathbb{C} $. 

In [ ]:
random.seed(1)
# Load data
data = np.loadtxt('data/pentagonsamplesSmall2.txt', delimiter=',')

print(data.shape[0])

# Energy computation
data_aug = np.zeros((data.shape[0], 10)) # augmented data
data_aug[:,2] = 1 
data_aug[:,[8,9,4,5,6,7]] = data
angle_data = np.zeros((data.shape[0],5)) # internal angles

for i in range(5):
    ii = (2*i -2)%10
    jj = (2*i - 1)%10
    u = data_aug[:, [ii, jj]]
    
    ii = (2*i)%10
    jj = (2*i + 1)%10
    v = data_aug[:, [ii , jj]]
    
    ii = (2*i + 2)%10
    jj = (2*i + 3)%10
    w = data_aug[:, [ii, jj]]
    
    z = np.sum((u - v)*(w-v), axis = 1)
    
    angle_data[:,i]  = np.arccos( np.minimum(z,1) )
    
energy = np.sum((angle_data - (3/5)*np.pi)**2 , axis = 1)

# plot sample pentagons
plt.figure(figsize=(14,6))
for i in range(60):
    ind = random.randint(0, data.shape[0])
    x = data_aug[ind].reshape(5,2).T
    cent = np.mean(x,axis=1)
    plt.subplot(6,10,i+1)
    plt.plot(x[0,range(-1,5)], x[1,range(-1,5)] , '-o', c= cm.magma(energy[ind]/np.max(energy)))
    plt.axis('square')
    plt.axis('off')
    plt.xlim(cent[0]-1.3,cent[0]+1.3);
    plt.ylim(cent[1]-1.3,cent[1]+1.3);

In [ ]:
data_aug[0]

In [ ]:
#PCA
pca = PCA(n_components=5)
data_pca = pca.fit(data).transform(data)

print('Explained Variance', pca.explained_variance_ratio_)

fig = go.Figure(data=[go.Scatter3d(
    x=data_pca[:,0], y=data_pca[:,1], z=data_pca[:,2], 
    mode ='markers', 
    marker=dict(size = 3, color =energy, colorscale='plasma' , opacity = 1)
)])


fig.update_layout(autosize=False, width=700, height=700)  

fig.show()

In [ ]:
# ISOMAP
# random subsample
ind_rand_data = np.random.randint(0, high=data.shape[0], size= 4000)
rand_data = data[ind_rand_data , :]

iso = Isomap(n_components = 3 , n_neighbors = 7)
rand_data_iso = iso.fit_transform(rand_data)

fig = go.Figure(data=[go.Scatter3d(
    x=rand_data_iso[:,0], y=rand_data_iso[:,1], z=rand_data_iso[:,2], 
    mode ='markers', 
    marker=dict(size = 3, color =energy[ind_rand_data], colorscale='plasma')
)])

fig.update_layout(autosize=False, width=700, height=700)

fig.show()

In [ ]:
# LLE
# random subsample
ind_rand_data = np.random.randint(0, high=data.shape[0], size= 8000)
rand_data = data[ind_rand_data , :]

rand_data_lle = LLE(n_components = 3 , n_neighbors = 20).fit_transform(rand_data)

fig = go.Figure(data=[go.Scatter3d(
    x=rand_data_lle[:,0], y=rand_data_lle[:,1], z=rand_data_lle[:,2], 
    mode ='markers', 
    marker=dict(size = 3, color =energy[ind_rand_data], colorscale='plasma')
)])

fig.update_layout(autosize=False, width=700, height=700)

fig.show()

In [ ]:
# Persistence Computation
n_land = 500
res = ripser(data, maxdim=2, n_perm = n_land, coeff =13, thresh = 1.8)
dgms = res['dgms']
plt.figure(figsize = (4,4)) 
plot_diagrams(dgms)

In [ ]:
pers1 = dgms[1][:,1] - dgms[1][:,0]
pers2 = dgms[2][:,1] - dgms[2][:,0]

pers1_sorted = np.sort(pers1)[::-1]
pers2_sorted = np.sort(pers2[pers2 < np.Inf])[::-1]

print('Persistence of top 10 1-d features:\n', np.around(pers1_sorted[0:10],decimals=2), '\n')
print('Persistence of top 10 2-d features:\n', np.around(pers2_sorted[0:10],decimals=2))

In [ ]:
n_lands = 1000

cc = CircularCoords(data , n_landmarks= n_lands)

In [ ]:
coho_classes = [0, 1, 3, 4 ]

circular_coords = []

for i in coho_classes:
    circular_coords.append(cc.get_coordinates(perc = 0.6, cocycle_idx=i, standard_range= False))


In [ ]:
thetas= CircleMapUtils.linear_combination(circular_coords, [[1,1,0,0], [0,1,0,0],[0,0,1,1]])

theta1 = CircleMapUtils.center(thetas[0])
theta2 = CircleMapUtils.center(thetas[1])
theta3 = CircleMapUtils.center(thetas[2])


# Plot the data using the computed circular coordintaes, and colors given by pentagon energy
fig = go.Figure(data=[go.Scatter3d(
    x=theta1, y=theta2, z=theta3, 
    mode ='markers', 
    marker=dict(size = 3, color =energy, colorscale='plasma')
)])

fig.update_layout(autosize=False, width=700, height=700)

fig.show()

---

<img src="https://i.ibb.co/86WnYMT/Eq-Penta-Surf.png" alt="test-1" border="0" width=700px>

In [ ]:
#plot with hovering affects
#test with 64 enconding

fig = go.Figure(data=[
    go.Scatter3d(x=theta1, y=theta2, z=theta3, 
        mode='markers', marker=dict(size = 2, color =energy, colorscale='plasma')
    )
])

# turn off native plotly.js hover effects - make sure to use
# hoverinfo="none" rather than "skip" which also halts events.
fig.update_traces(hoverinfo="none", hovertemplate=None)
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,8],),
        yaxis = dict(range=[-1,8],),
        zaxis = dict(range=[-1,8],),
    ), width=700, height=700
)

app = JupyterDash(__name__)

server = app.server

app.layout = html.Div([
    dcc.Graph(id="graph-basic-2", figure=fig, clear_on_unhover=True),
    dcc.Tooltip(id="graph-tooltip"),
])


@app.callback(
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("graph-basic-2", "hoverData"),
)

def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    # demo only shows the first point, but other points may also be available
    pt = hoverData["points"][0]
    bbox = pt["bbox"]
    num = pt["pointNumber"]

    img_fname = 'images/'+str(num+1)+'.png'
    encoded_img = base64.b64encode(open(img_fname, 'rb').read())

    children = [
        html.Div([
            html.Img(src='data:image/png;base64,{}'.format(encoded_img.decode()), style={'width': '100%'}),
        ], style={'width': '80px', 'white-space': 'normal'})
    ]

    return True, bbox, children

app.run_server(mode="inline")


----